In [2]:
import os
import sys 
sys.path.append("/home/binhng/Workspace/robocasa/robocasa")

In [32]:
import collections
import dataclasses
import logging
import math
import pathlib

import imageio

import numpy as np
# from openpi_client import image_tools
# from openpi_client import websocket_client_policy as _websocket_client_policy
import tqdm
# import tyro
# import robomimic.utils.file_utils as FileUtils
# import robocasa.utils.model_zoo.file_utils as FileUtils
from robocasa.utils.robomimic.robomimic_dataset_utils import get_env_metadata_from_dataset
# import robomimic.utils.env_utils as EnvUtils
import robocasa.utils.robomimic.robomimic_env_utils as EnvUtils
# import robomimic.utils.obs_utils as ObsUtils
import robocasa.utils.robomimic.robomimic_obs_utils as ObsUtils

import robocasa

In [1]:
import h5py
import json

f = h5py.File('/home/binhng/Workspace/robocasa/robocasa/datasets/robocasa_30_demos_debug/PnPCabToCounter.hdf5')
f['data'].keys()

<KeysViewHDF5 ['demo_1181', 'demo_1217', 'demo_1236', 'demo_1251', 'demo_1518', 'demo_1585', 'demo_1711', 'demo_1801', 'demo_1902', 'demo_196', 'demo_2127', 'demo_2128', 'demo_2157', 'demo_2274', 'demo_2543', 'demo_2630', 'demo_2684', 'demo_2995', 'demo_320', 'demo_328', 'demo_361', 'demo_436', 'demo_453', 'demo_522', 'demo_61', 'demo_758', 'demo_804', 'demo_825', 'demo_902', 'demo_930']>

In [17]:
f['data']['demo_1217'].keys()

<KeysViewHDF5 ['action_dict', 'actions', 'actions_abs', 'dones', 'obs', 'rewards', 'states']>

In [20]:
f['data']['demo_1181']['action_dict'].keys()

<KeysViewHDF5 ['abs_pos', 'abs_rot_6d', 'abs_rot_axis_angle', 'gripper', 'rel_pos', 'rel_rot_6d', 'rel_rot_axis_angle']>

In [ ]:

obs_keys = [
    "robot0_joint_pos",
    "robot0_joint_pos_cos",
    "robot0_joint_pos_sin",
    "robot0_joint_vel",
    "robot0_joint_acc",
    "robot0_eef_pos",
    "robot0_eef_quat",
    "robot0_eef_quat_site",
    "robot0_gripper_qpos",
    "robot0_gripper_qvel",
    "robot0_base_pos",
    "robot0_base_quat",
    "robot0_base_to_eef_pos",
    "robot0_base_to_eef_quat",
    "robot0_base_to_eef_quat_site",
    "robot0_agentview_left_image",
    "robot0_agentview_left_depth",
    "robot0_agentview_left_segmentation_instance",
    "robot0_agentview_right_image",
    "robot0_agentview_right_depth",
    "robot0_agentview_right_segmentation_instance",
    "robot0_eye_in_hand_image",
    "robot0_eye_in_hand_depth",
    "robot0_eye_in_hand_segmentation_instance",
    "obj_pos",
    "obj_quat",
    "obj_to_robot0_eef_pos",
    "obj_to_robot0_eef_quat",
    "distr_counter_pos",
    "distr_counter_quat",
    "distr_counter_to_robot0_eef_pos",
    "distr_counter_to_robot0_eef_quat",
    "distr_cab_pos",
    "distr_cab_quat",
    "distr_cab_to_robot0_eef_pos",
    "distr_cab_to_robot0_eef_quat",
    "robot0_proprio-state",
    "object-state",
]
len(obs_keys)


38

In [34]:
demo = f["data"]["demo_1181"]                        # access demo 5
obs = demo["obs"]                                 # obervations across all timesteps
left_img = obs["robot0_agentview_left_image"][:]  # get left camera images in numpy format
ep_meta = json.loads(demo.attrs["ep_meta"])       # get meta data for episode
lang = ep_meta["lang"]                            # get language instruction for episode
f.close()

In [37]:
# ep_meta

In [38]:
from robocasa.utils.dataset_registry import SINGLE_STAGE_TASK_DATASETS, MULTI_STAGE_TASK_DATASETS
from robocasa.utils.dataset_registry import get_ds_path

# iterate through all atomic and composite tasks
for task in list(SINGLE_STAGE_TASK_DATASETS): # + list(MULTI_STAGE_TASK_DATASETS):
    print(f"Datasets for {task}")
    # human_path, ds_meta = get_ds_path(task=task, ds_type="human_im", return_info=True)  # human dataset path
    # horizon = ds_meta["horizon"]                                      # get suggested for dataset
    mg_path, ds_meta = get_ds_path(task=task, ds_type="mg_im", return_info=True)        # MimicGen dataset path
    # print(f"Human ds: {human_path}")
    print(f"MimicGen ds: {mg_path}")
    print(f"Dataset metadata:", ds_meta)
    # print(f"Dataset horizon:", horizon)
    print()

Datasets for PnPCounterToCab
MimicGen ds: /home/binhng/Workspace/robocasa/robocasa/datasets/v0.1/single_stage/kitchen_pnp/PnPCounterToCab/mg/2024-05-04-22-12-27_and_2024-05-07-07-39-33/demo_gentex_im128_randcams.hdf5
Dataset metadata: {'url': 'https://utexas.box.com/shared/static/7y9csrcx6uhhq4p3yctmm2df3rjqpw6g.hdf5', 'horizon': 500}

Datasets for PnPCabToCounter
MimicGen ds: /home/binhng/Workspace/robocasa/robocasa/datasets/v0.1/single_stage/kitchen_pnp/PnPCabToCounter/mg/2024-07-12-04-33-29/demo_gentex_im128_randcams.hdf5
Dataset metadata: {'url': 'https://utexas.box.com/shared/static/mfltre5w8qi8ps1fqicblx40p008425z.hdf5', 'horizon': 500}

Datasets for PnPCounterToSink
MimicGen ds: /home/binhng/Workspace/robocasa/robocasa/datasets/v0.1/single_stage/kitchen_pnp/PnPCounterToSink/mg/2024-05-04-22-14-06_and_2024-05-07-07-40-17/demo_gentex_im128_randcams.hdf5
Dataset metadata: {'url': 'https://utexas.box.com/shared/static/5vkb2fpthnw554q888fynk3zqwsgk2e5.hdf5', 'horizon': 700}

Datasets

In [18]:

ROBOCASA_DUMMY_ACTION = [0.0] * 6 + [-1.0] + [0.0] * 4 + [-1.0]
LIBERO_ENV_RESOLUTION = 256  # resolution used to render training data
DEFAULT_EVAL_UPDATE_KWARGS =  {
    "generative_textures": None,
    "randomize_cameras": False,
    "obj_instance_split": "B",
    "layout_ids": None,
    "style_ids": None,
    "scene_split": None,
    "layout_and_style_ids": [
                    [
                        1,
                        1
                    ],
                    [
                        2,
                        2
                    ],
                    [
                        4,
                        4
                    ],
                    [
                        6,
                        9
                    ],
                    [
                        7,
                        10
                    ]
                ],
    "camera_heights": 256,
    "camera_widths": 256,
}


SHAPE_META = {
    "obs": {
        "robot0_agentview_left_image": {
            "shape": [256, 256,3],
            "type": "rgb"
        },
        "robot0_eye_in_hand_image": {
            "shape": [256, 256, 3],
            "type": "rgb"
        },
        "robot0_agentview_right_image": {
            "shape": [256, 256, 3],
            "type": "rgb"
        },
        "robot0_base_to_eef_pos": {
            "shape": [3]
            # type default: low_dim (not explicitly listed)
        },
        "robot0_base_to_eef_quat": {
            "shape": [4]
        },
        "robot0_gripper_qpos": {
            "shape": [2]
        },
    },
    "action": {
        "shape": [12]
    }
}


In [44]:
env_meta = {
    "env_name": "PnPCabToCounter",
    "type": 1
}
def _get_robocasa_env(env_meta, shape_meta, enable_render=True):
    """Initializes and returns the LIBERO environment, along with the task description."""
    modality_mapping = collections.defaultdict(list)
    for key, attr in shape_meta['obs'].items():
        modality_mapping[attr.get('type', 'low_dim')].append(key)
    ObsUtils.initialize_obs_modality_mapping_from_dict(modality_mapping)
    env_name = env_meta['env_name']
    env_name = env_name[3:] if env_name.startswith('MG_') else env_name
    env = EnvUtils.create_env_from_metadata(
        env_meta=env_meta,
        env_name=env_name,
        render=False, 
        render_offscreen=enable_render,
        use_image_obs=enable_render, 
    )
    return env

In [ ]:
# from robocasa.utils.env_utils 
# import create_env  
# from robocasa.scripts.playback_dataset 
# import reset_to  
# # Create environment  
# env = create_env(env_name="KitchenPnPCounterToSink-v0") 
# # # Reset to specific state  
# state = {"states": flattened_state_array,  # numpy array from dataset      
# "model": mujoco_xml_string,      # optional model file      "ep_meta": episode_metadata      # optional episode metadata  }  reset_to(env, state)
 

In [45]:
_get_robocasa_env(env_meta, SHAPE_META, enable_render=True)

KeyError: 'env_kwargs'

In [ ]:
# data_path = "/home/binhng/Workspace/robocasa/robocasa/datasets/robocasa_30_demos_debug/PnPCabToCounter.hdf5"
# env_meta = get_env_metadata_from_dataset(data_path)

KeyError: "Unable to synchronously open attribute (can't locate attribute: 'env_args')"

In [51]:
import h5py 

data_path = '/home/binhng/Workspace/robocasa/robocasa/datasets/robocasa_30_demos_debug/PnPCabToCounter.hdf5'
data = h5py.File(data_path, 'r')


In [ ]:
model_xml = data["data/demo_1181"].attrs["model_file"]

In [92]:
model_xml

'<mujoco model="base">\n  <compiler angle="radian" meshdir="meshes/"/>\n  <option impratio="20" density="1.2" viscosity="2e-05" cone="elliptic"/>\n  <size njmax="5000" nconmax="5000"/>\n  <visual>\n    <map znear="0.001"/>\n  </visual>\n  <asset>\n    <texture type="skybox" builtin="flat" rgb1="1 1 1" rgb2="1 1 1" width="256" height="1536"/>\n    <texture type="2d" name="wall_room_wall" file="/home/abhim/robocasa/robocasa/robocasa/models/assets/textures/tiles/diamond_tiles.png"/>\n    <texture type="2d" name="wall_backing_room_wall" file="/home/abhim/robocasa/robocasa/robocasa/models/assets/textures/flat/light_gray.png"/>\n    <texture type="2d" name="wall_left_room_wall" file="/home/abhim/robocasa/robocasa/robocasa/models/assets/textures/tiles/diamond_tiles.png"/>\n    <texture type="2d" name="wall_left_backing_room_wall" file="/home/abhim/robocasa/robocasa/robocasa/models/assets/textures/flat/light_gray.png"/>\n    <texture type="2d" name="wall_right_room_wall" file="/home/abhim/robo

In [82]:
init_state = data['data']['demo_1181']['states'][:][0][:]

In [83]:
data['data']['demo_1181']['states'][:].shape

(319, 120)

In [ ]:
# data['data'].attrs['env_args']

KeyError: "Unable to synchronously open attribute (can't locate attribute: 'env_args')"

In [84]:
# init_state
ep_meta = json.loads(data['data']['demo_1181'].attrs["ep_meta"])  

In [ ]:
from robocasa.utils.env_utils import create_env  
from robocasa.scripts.playback_dataset import reset_to  
import robosuite

# Create environment  
env = create_env(env_name="PnPCabToCounter")  

# # Reset to specific state  


[robosuite INFO] Loading controller configuration from: /home/binhng/Workspace/robocasa/robosuite/robosuite/controllers/config/robots/default_pandaomron.json (composite_controller_factory.py:121)


In [86]:
init_state[0].shape

()

In [ ]:
env = robosuite.make(**env_kwargs)

In [93]:
state = {  
    "states": init_state,  # numpy array from dataset  
    "model": model_xml,      # optional model file  
    # "ep_meta": ep_meta     # optional episode metadata  
}  
reset_to(env, state)

ValueError: No "geom" with name counter_corner_main_main_group_base_right_visual exists. Available "geom" names = ('mobilebase0_g0_col', 'mobilebase0_fixed_support', 'mobilebase0_g1_col', 'mobilebase0_support', 'robot0_g0_vis', 'robot0_g1_vis', 'robot0_g2_vis', 'robot0_g3_vis', 'robot0_g4_vis', 'robot0_g5_vis', 'robot0_g6_vis', 'robot0_g7_vis', 'robot0_g8_vis', 'robot0_g9_vis', 'robot0_g10_vis', 'robot0_g11_vis', 'robot0_link0_collision', 'robot0_g12_vis', 'robot0_link1_collision', 'robot0_g13_vis', 'robot0_link2_collision', 'robot0_g14_vis', 'robot0_g15_vis', 'robot0_g16_vis', 'robot0_g17_vis', 'robot0_link3_collision', 'robot0_g18_vis', 'robot0_g19_vis', 'robot0_g20_vis', 'robot0_g21_vis', 'robot0_link4_collision', 'robot0_g22_vis', 'robot0_g23_vis', 'robot0_g24_vis', 'robot0_link5_collision', 'robot0_g25_vis', 'robot0_g26_vis', 'robot0_g27_vis', 'robot0_g28_vis', 'robot0_g29_vis', 'robot0_g30_vis', 'robot0_g31_vis', 'robot0_g32_vis', 'robot0_g33_vis', 'robot0_g34_vis', 'robot0_g35_v